# Task 4
通过评分与短评数据，构建情感褒贬分析分类器，通过短评数据预测用户“喜欢”or“不喜欢”电影

In [4]:
import pandas as pd
import numpy as np
data = pd.read_csv('./data/cmt_data2.csv', lineterminator='\n')
data.head()

,user,star,comment,time,name_length,cmt_length,year,month,day,weekday,weeknumber,name_w_alpha,cmt_w_alpha,name_w_digit,cmt_w_digit
0,kingfish,5.0,不需要女主角的好电影,2006-03-22,8,10,2006,3,22,3,12,1,0,0,0
1,犀牛,5.0,当年的奥斯卡颁奖礼上，被如日中天的《阿甘正传》掩盖了它的光彩，而随着时间的推移，这部电影在越...,2005-10-28,2,160,2005,10,28,5,43,0,0,0,0
2,711|湯好运,5.0,策划了19年的私奔……,2010-03-27,7,11,2010,3,27,6,12,0,0,1,1
3,Eve|Classified,5.0,“这是一部男人必看的电影。”人人都这么说。但单纯从性别区分，就会让这电影变狭隘。《肖申克的救...,2008-05-09,14,218,2008,5,9,5,19,1,0,0,0
4,艾小柯,5.0,关于希望最强有力的注释。,2006-06-20,3,12,2006,6,20,2,25,0,0,0,0


In [5]:
data.star.unique()

array([  5.,   2.,   4.,   3.,  nan,   1.])

In [6]:
data.star.value_counts()

5.0    9735
4.0    4915
3.0    1609
2.0     291
1.0     141
Name: star, dtype: int64

In [7]:
## star缺失值处理
#看均值
data['star'].mean()

4.4266371098196631

In [8]:
#看mode
data['star'].value_counts().head(1)

5.0    9735
Name: star, dtype: int64

【初步结论】<br>
不应该用mean或mode填充，因为没打分的不一定是喜欢。<br>
保持原样或者用3星填充 --> 不会影响情感分类，因为大于等于4算“喜欢”，小于等于2算“不喜欢”

In [9]:
# 用3星填充
data['star'].fillna(3, inplace=True)
data['star'].value_counts()

5.0    9735
4.0    4915
3.0    2018
2.0     291
1.0     141
Name: star, dtype: int64

In [19]:
# 加情感分类标签
def addTag(star):
    tag = np.nan
    if star >= 4:
        tag = 1
    elif star <= 2:
        tag = 0
    return tag

In [20]:
data['tag'] = data['star'].map(lambda x: addTag(x))

In [21]:
data.head()

,user,star,comment,time,name_length,cmt_length,year,month,day,weekday,weeknumber,name_w_alpha,cmt_w_alpha,name_w_digit,cmt_w_digit,tag
0,kingfish,5.0,不需要女主角的好电影,2006-03-22,8,10,2006,3,22,3,12,1,0,0,0,1.0
1,犀牛,5.0,当年的奥斯卡颁奖礼上，被如日中天的《阿甘正传》掩盖了它的光彩，而随着时间的推移，这部电影在越...,2005-10-28,2,160,2005,10,28,5,43,0,0,0,0,1.0
2,711|湯好运,5.0,策划了19年的私奔……,2010-03-27,7,11,2010,3,27,6,12,0,0,1,1,1.0
3,Eve|Classified,5.0,“这是一部男人必看的电影。”人人都这么说。但单纯从性别区分，就会让这电影变狭隘。《肖申克的救...,2008-05-09,14,218,2008,5,9,5,19,1,0,0,0,1.0
4,艾小柯,5.0,关于希望最强有力的注释。,2006-06-20,3,12,2006,6,20,2,25,0,0,0,0,1.0


In [27]:
train_val = data[['comment', 'tag']].dropna(how='any')
train_val.head()

,comment,tag
0,不需要女主角的好电影,1.0
1,当年的奥斯卡颁奖礼上，被如日中天的《阿甘正传》掩盖了它的光彩，而随着时间的推移，这部电影在越...,1.0
2,策划了19年的私奔……,1.0
3,“这是一部男人必看的电影。”人人都这么说。但单纯从性别区分，就会让这电影变狭隘。《肖申克的救...,1.0
4,关于希望最强有力的注释。,1.0


In [29]:
# stopword list
stopwords = []
with open('stopwords.txt', 'r', errors='ignore') as f:        
    for line in f.readlines():
        stopwords.append(line.strip())

In [32]:
# 中文comment 分词 + 去除停用词
import jieba 
def sent2word(sentence):
    """
    Segment a chinese sentence to words
    Delete stopwords
    """
    segList = jieba.cut(sentence)
    segResult = []
    for w in segList:
        segResult.append(w)

               
    words = []
    for word in segResult:
        if word in stopwords:
            # print "stopword: %s" % word
            continue
        else:
            words.append(word)

    return ' '.join(words)

In [33]:
train_val['seg_cmt'] = train_val['comment'].map(lambda x: sent2word(x))
train_val.head()

,comment,tag,seg_cmt
0,不需要女主角的好电影,1.0,女主角 电影
1,当年的奥斯卡颁奖礼上，被如日中天的《阿甘正传》掩盖了它的光彩，而随着时间的推移，这部电影在越...,1.0,当年 奥斯卡 颁奖礼 如日中天 阿甘正传 掩盖 光彩 时间 推移 这部 电影 越来越 心中 ...
2,策划了19年的私奔……,1.0,策划 19 年 私奔
3,“这是一部男人必看的电影。”人人都这么说。但单纯从性别区分，就会让这电影变狭隘。《肖申克的救...,1.0,这是 一部 男人 必看 电影 说 性别 区分 电影 变 狭隘 肖申克 救赎 突破 男人 电影...
4,关于希望最强有力的注释。,1.0,希望 强有力 注释


In [34]:
train_val.to_csv('./data/train_val.csv')

In [35]:
# 用sklearn.feature_extraction.text.TfidfVectorizer
X_all = train_val['seg_cmt'].values.tolist()
X_all[:100]

['女主角 电影',
 '当年 奥斯卡 颁奖礼 如日中天 阿甘正传 掩盖 光彩 时间 推移 这部 电影 越来越 心中 地位 超越 阿甘 现实 令 疲惫 无力感 翻出 这张 碟 重获 力量 毫无疑问 本片 位列 男人 必看 电影 前三名 回顾 一段 经典台词 羽翼 光辉 世界 黑暗 牢狱 长久 围困',
 '策划 19 年 私奔',
 '这是 一部 男人 必看 电影 说 性别 区分 电影 变 狭隘 肖申克 救赎 突破 男人 电影 局限 通篇 充满 令人 难以置信 温馨 基调 电影 里 主题 希望 \r \r\n 无奈 肖申克 囚禁 心灵 自由 那种 囹圄 无奈 布鲁克 灰心 瑞德 智慧 安迪 智慧 信任 希望 勇敢 面对 恐惧心理 打败 \r \r\n 经典 电影 经典 做 一件 事 角度 欣赏 希望 美好',
 '希望 强有力 注释',
 '种鸟 关不住',
 '忒 经典 东西 我要 坟墓',
 '超级 喜欢 超级 喜欢 不看 人生 圆满',
 '无疑 一部 经典 爱死',
 '真的 喜欢 好看 感觉',
 '生命 是从 洞穴 通往 世界 世界 雨 中 颤抖 i   hope',
 'Fear   Can   Hold   You   Prisoner   Hope   Can   Set   You   Free',
 'Hope   is   a   good   thing   and   maybe   the   best   thing   of   all',
 '人会 喜欢 书 电影',
 'hope   is   a   good   thing',
 '越狱 感觉 改编自',
 '1994 年 台湾 引进 一部 卖座 老片 The   Sting 错 译成 刺激 1995 年 本片 上映 时 片商 剧情 刺激 类似 地方 高智商 复仇 译成 刺激 1995 1998 年 一部 片子 Return   To   Paradise 含有 牢狱 情节 译成 刺激 1998',
 '大众 经典 感冒 欣赏 水平 不行',
 '',
 '一部 爱情 美女 电影 光芒四射',
 '男人 舍得 背叛',
 '完 振奋',
 '墙里 喊 自由 墙外',
 '喜欢 电影',
 '被誉为 影史 最棒 排名 肯定 经典 缜密 逃狱 现代版 基督山 恩仇记 弗兰克 德拉

In [36]:
from sklearn.feature_extraction.text import TfidfVectorizer as TFIV
# 初始化TFIV对象，再去英语停用词，加2元语言模型http://blog.csdn.net/han_xiaoyang/article/details/50629608
tfv = TFIV(min_df=3, max_features=None, ngram_range=(1, 2), use_idf=1, smooth_idf=1, sublinear_tf=1, stop_words = 'english')

tfv.fit(X_all)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=3,
        ngram_range=(1, 2), norm='l2', preprocessor=None, smooth_idf=1,
        stop_words='english', strip_accents=None, sublinear_tf=1,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=1,
        vocabulary=None)

In [38]:
X = tfv.transform(X_all)

In [39]:
X

<15081x16146 sparse matrix of type '<class 'numpy.float64'>'
	with 198575 stored elements in Compressed Sparse Row format>

In [40]:
y = train_val['tag']

In [43]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, random_state=1234)

In [45]:
# 多项式朴素贝叶斯
from sklearn.naive_bayes import MultinomialNB as MNB

model_NB = MNB()
model_NB.fit(X_train, y_train) 
MNB(alpha=1.0, class_prior=None, fit_prior=True)

from sklearn.cross_validation import cross_val_score
print("多项式贝叶斯分类器20折交叉验证得分: ", np.mean(cross_val_score(model_NB, X_train, y_train, cv=20, scoring='roc_auc')))
# 多项式贝叶斯分类器20折交叉验证得分: 0.950837239

多项式贝叶斯分类器20折交叉验证得分:  0.661586237719


### 得分低，表明模型不够general

In [53]:
# test the model on the testing set, and check the accuracy

from sklearn import metrics
y_pred = model_NB.predict(X_val)
print(metrics.accuracy_score(y_val, y_pred))
print(metrics.confusion_matrix(y_val, y_pred))
print(metrics.classification_report(y_val, y_pred))

0.968443383718
[[   0  119]
 [   0 3652]]
             precision    recall  f1-score   support

        0.0       0.00      0.00      0.00       119
        1.0       0.97      1.00      0.98      3652

avg / total       0.94      0.97      0.95      3771



/opt/conda/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


## 没有预测任何差评？
- 在预测差评方面表现不好，可能原因是数据中差评的量相对太少，样本是高度bias。
  - 数据出现这种情况是因为取的是排名前250的影片评价 --> 评价高<br>
- 在这种数据情况下，我怀疑选更fancy的模型的作用不会很大<br>
- 可以进一步尝试
  - 不分validation data，数据全部用于训练，evaluation用CV
  - downsampling或upsampling
  - ……